In [ ]:
import numpy as np
import pandas as pd
import spacy
import scipy as spy
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
!python -m spacy download it_core_news_lg

In [ ]:
nlp = spacy.load("it_core_news_lg")

Calcolo dell'IAA sulle annotazioni:

1.   IAA sul GS Object
2.   IAA sul Defaulting

In [ ]:
#annotazione manuale

file = pd.read_excel(r'/content/drive/MyDrive/IOcompletion_tesi_agnese_daffara/Dataset/IMPLICIT_results.xlsx')
predicted1 = file['obj_1'][:600]
predicted2 = file['obj_2'][:600]

In [ ]:
#cosine similarity tra annotatori sul GS Object
couples = zip(predicted1, predicted2)

#definisco la funzione per calcolare la similarity
def calculate_similarity(word1, word2):
  word1_token = nlp(word1)
  word2_token = nlp(word2)
  if word2 == "unknown" or word1 == "unknown":
      return 0
  if word1_token.has_vector and word2_token.has_vector:
      similarity = word1_token.similarity(word2_token)
      return round(similarity,2)
  else:
      return 0

similarity_scores = []
for couple in couples:
  similarity = calculate_similarity(couple[0], couple[1])
  similarity_scores.append(similarity)

print('similarity tra annotatori =', similarity_scores)
print('media tra annotatori =', np.mean(similarity_scores))

Calcolo dell'IAA su GS Object LEMMATIZZATI

In [ ]:
#annotazione manuale lemmatizzata

file = pd.read_excel(r'/content/drive/MyDrive/IOcompletion_tesi_agnese_daffara/Dataset/IMPLICIT_results_lemmatized.xlsx')
predicted1 = file['obj_1'][:600]
predicted2 = file['obj_2'][:600]

In [ ]:
#cosine similarity tra annotatori sul GS Object
couples = zip(predicted1, predicted2)

#definisco la funzione per calcolare la similarity
def calculate_similarity(word1, word2):
  word1_token = nlp(word1)
  word2_token = nlp(word2)
  if word2 == "unknown" or word1 == "unknown":
      return 0
  if word1_token.has_vector and word2_token.has_vector:
      similarity = word1_token.similarity(word2_token)
      return round(similarity,2)
  else:
      return 0

similarity_scores = []
for couple in couples:
  similarity = calculate_similarity(couple[0], couple[1])
  similarity_scores.append(similarity)

print('similarity tra annotatori =', similarity_scores)
print('media tra annotatori =', np.mean(similarity_scores))

In [ ]:
from sklearn.metrics import cohen_kappa_score

#annotazione manuale del defaulting
def1 = file['defaulting_1'][:600]
def2 = file['defaulting_2'][:600]

mapping = {"Lexical Defaulting": 0, "Pragmatic Defaulting": 1}
def1n = [mapping[a] for a in def1]
def2n = [mapping[a] for a in def2]

cohen_kappa = cohen_kappa_score(def1n, def2n)
print(f'Cohen\'s Kappa: {cohen_kappa}')

In [ ]:
verbi = ['ascoltare', 'attendere', 'bere', 'cantare', 'chiamare', 'combattere', 'condurre', 'consumare', 'correre', 'cucinare', 'dirigere', 'disegnare', 'fumare', 'giocare', 'guadagnare', 'guidare', 'leggere', 'mangiare', 'ordinare', 'pagare', 'perdere', 'pregare', 'preoccupare', 'provare', 'respirare', 'scrivere', 'servire', 'suonare', 'tirare', 'vincere']
gruppi1 = [def1[i:i+20] for i in range(0, len(def1), 20)]
gruppi2 = [def2[i:i+20] for i in range(0, len(def2), 20)]

nuovo = zip(verbi, gruppi1, gruppi2) #ho creato un oggetto che associa ogni gruppo di 20 annotazioni del primo e del secondo annotatore ad un verbo
#nuovo = zip(verbi, gruppi2)

dati = []

for i in nuovo: #per ogni elemento nell'oggetto, conto quante volte è stato annotato Lexical Defaulting e quante "Pragmatic"
  lde = 0
  pde = 0
  for p in i:
    for el in p:
      if "Lexical Defaulting" in el:
        lde = lde + 1
      if "Pragmatic Defaulting" in el:
        pde = pde + 1
  dato = (i[0], lde, pde)
  dati.append(dato)

df = pd.DataFrame({'verbo': [dato[0] for dato in dati], 'Lexical Defaulting': [dato[1] for dato in dati], 'Pragmatic Defaulting': [dato[2] for dato in dati]})
print(df)
df = df.sort_values(by='Lexical Defaulting')

df.set_index('verbo', inplace=True)

#creo il grafico a barre empilato con Seaborn
sns.set(style='whitegrid', rc={"axes.grid.axis": "y", "axes.grid.which": "both", "ytick.left": True})
sns.set_context("notebook", rc={"ytick.major.pad": 0.5})
sns.set(style='whitegrid')
ax = df.plot(kind='barh', stacked=True, figsize=(12, 8), colormap='viridis')
plt.title("Distribuzione del tipo di Defaulting tra i verbi\n(IMPLICIT dataset: 2 annotatori)")
#plt.title("Distribuzione del tipo di Defaulting tra i verbi\n(IMPLICIT dataset: secondo annotatore)")
plt.xlabel('n. annotazioni per tipo')
plt.ylabel('')
plt.legend(title='', loc='lower right', bbox_to_anchor=(1, 0.0), borderaxespad=0.)
plt.xlim(0, max(df.sum(axis=1)))
plt.xticks(range(1, 41))
#plt.xticks(range(1, 21))
plt.show()

df

In [ ]:
#calcolo il range di Oggetti per verbo nell'annotazione manuale

verbi = ['ascoltare', 'attendere', 'bere', 'cantare', 'chiamare', 'combattere', 'condurre', 'consumare', 'correre', 'cucinare', 'dirigere', 'disegnare', 'fumare', 'giocare', 'guadagnare', 'guidare', 'leggere', 'mangiare', 'ordinare', 'pagare', 'perdere', 'pregare', 'preoccupare', 'provare', 'respirare', 'scrivere', 'servire', 'suonare', 'tirare', 'vincere']

#creo una funzione che divide ogni lista in gruppi da 20 scores ciascuno, ogni gruppo si riferisce ad un verbo
def verb_values(score):
  gruppi = [score[i:i+20] for i in range(0, len(score), 20)]
  lista = list(zip(verbi, gruppi))
  return lista

#creo una variabile che contiene le due liste di Oggetti GS
ress = [predicted1, predicted2]
#creo una lista che contiene tutti i risultati divisi in gruppi
results = []
for res in ress:
  results.append(verb_values(res))
#creo un dizionario che contiene i verbi e i punteggi che hanno ottenuto in ogni lista
d={}
for verbo in verbi:
  d[verbo] = []
#itero su ogni lista di risultati
for l in results:
  for item in l:
    #appendo alla value del verbo tutti gli score dei modelli per quel verbo
    d[item[0]].append(item[1])

#definisco una funzione per contare le parole uniche per ogni verbo
def conta_parole_uniche(d):
    risultati = {}
    for verbo, liste_parole in d.items():
        parole_uniche = set()
        for serie in liste_parole:
            parole_uniche.update(serie)
        numero_parole_uniche = len(parole_uniche)
        risultati[verbo] = numero_parole_uniche
    return risultati

#applico la funzione al dizionario
risultati = conta_parole_uniche(d)

#creo il dataframe ordinato
df = pd.DataFrame(list(risultati.items()), columns=['Verbo', 'Range'])
df_sorted = df.sort_values(by='Range', ascending=False)

#creo il grafico
sns.set_style("whitegrid")
plt.figure(figsize=(12, 8))
sns.barplot(x='Range', y='Verbo', data=df_sorted, palette='viridis')
plt.xlabel('')
plt.ylabel('')
plt.title("Range di Oggetti (IMPLICIT dataset: 2 annotatori)")
plt.xticks(range(0, int(df['Range'].max()) + 1, 1))
plt.show()